In [1]:
import pandas as pd
import pandas_profiling

data = pd.read_csv("C:\\Users\\KulkovaAV\\Desktop\\Feature engineering\\дз_корректное\\дз\\data\\aac_shelter_outcomes.csv")

In [2]:
#Информация о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78256 entries, 0 to 78255
Data columns (total 12 columns):
age_upon_outcome    78248 non-null object
animal_id           78256 non-null object
animal_type         78256 non-null object
breed               78256 non-null object
color               78256 non-null object
date_of_birth       78256 non-null object
datetime            78256 non-null object
monthyear           78256 non-null object
name                54370 non-null object
outcome_subtype     35963 non-null object
outcome_type        78244 non-null object
sex_upon_outcome    78254 non-null object
dtypes: object(12)
memory usage: 7.2+ MB


In [3]:
#Количество пропущенных значений по каждому столбцу
data.isnull().sum()

age_upon_outcome        8
animal_id               0
animal_type             0
breed                   0
color                   0
date_of_birth           0
datetime                0
monthyear               0
name                23886
outcome_subtype     42293
outcome_type           12
sex_upon_outcome        2
dtype: int64

Описание датафрейма
age_upon_outcome - время с последней выгрузки (= datetime - date_of_birth)
animal_id - идентификационный номер животного
animal_type - тип животного
breed - порода (домашняя, короткая шерсть, микс)
color - цвет
date_of_birth - дата рождения
datetime = monthyear - текущая дата?
name - имя
outcome_subtype - партнер, приемный, риск бешенства, страдающий, агрессивный, в питомнике, украден, и т.д.
outcome_type - вернулся к хозяину, умер, перевезен, взят из приюта, усыпление
sex_upon_outcome - пол животного (например, "неповрежденный самец", "стерилизованная самка")

In [4]:
# Нет ни одной записи, где datetime != monthyear => это два одинаковых столбца. Удалим один из них.
data[data.datetime != data.monthyear].head()
data = data.drop('monthyear', axis = 1)

In [5]:
#Количество пустых записей в столбце name = 23886. Нет смысла заполнять его какими-то записями, 
#т.к. для анализа - имя животного не важно.Следовательно, можем удалить данный столбец.
data = data.drop('name', axis = 1)

In [6]:
#Находим повторяющиеся animal_id и смотрим чем отличаются записи по данному животному. 
#Либо их объединить можно, если они различаются, либо удалить, если строки повтряются
grouped = data.groupby('animal_id').count().reset_index().rename(columns={'age_upon_outcome':'count'})[['animal_id', 'count']]
grouped[lambda x: x['count'] > 1].head(15)


,animal_id,count
0,A006100,3
28,A245945,2
52,A282897,2
56,A287017,2
72,A304036,2
77,A307010,2
90,A318574,2
94,A322813,2
101,A329502,2
116,A337686,2


In [7]:
data[data.animal_id == 'A666430']

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,outcome_subtype,outcome_type,sex_upon_outcome
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Partner,Transfer,Spayed Female


In [8]:
#Проанализировав датафрейм,получилось, что 14354 отсутствующие записи имеет тип "Вернулся к хозяину"
df = data[data.outcome_type == 'Return to Owner'][['outcome_subtype']]
df['outcome_subtype'].unique()
df.shape

(14354, 1)

In [9]:
#Предположила, что собаки, которые вернулись к хозяину вероятно были украдены. Заменила пропуски данным условием.
data.loc[data.outcome_type == 'Return to Owner', 'outcome_subtype'] = "Possible Theft"

In [10]:
#Количество пропущенных значений по каждому столбцу
data.isnull().sum()

age_upon_outcome        8
animal_id               0
animal_type             0
breed                   0
color                   0
date_of_birth           0
datetime                0
outcome_subtype     27939
outcome_type           12
sex_upon_outcome        2
dtype: int64

In [11]:
# Убрала неизвестные значения в столбце age_upon_outcome.Перевела в формат даты и оставила дни.  
# Т.к. age_upon_outcome = datetime - date_of_birth => можно убрать столбец datetime, к примеру.
from datetime import datetime
data['age_upon_outcome'] = data['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')) - data['date_of_birth'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
data['age_upon_outcome'] = data['age_upon_outcome'].apply(lambda x: x.days)
data = data.drop('datetime', axis = 1)

In [12]:
data.isnull().sum()

age_upon_outcome        0
animal_id               0
animal_type             0
breed                   0
color                   0
date_of_birth           0
outcome_subtype     27939
outcome_type           12
sex_upon_outcome        2
dtype: int64

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78256 entries, 0 to 78255
Data columns (total 9 columns):
age_upon_outcome    78256 non-null int64
animal_id           78256 non-null object
animal_type         78256 non-null object
breed               78256 non-null object
color               78256 non-null object
date_of_birth       78256 non-null object
outcome_subtype     50317 non-null object
outcome_type        78244 non-null object
sex_upon_outcome    78254 non-null object
dtypes: int64(1), object(8)
memory usage: 5.4+ MB


In [14]:
pandas_profiling.ProfileReport(data)

Number of variables,9
Number of observations,78256
Total Missing (%),4.0%
Total size in memory,5.4 MiB
Average record size in memory,72.0 B
Numeric,1
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [17]:
# Dataset имеет 23 повторяющиеся строки. Удаляем их.
data = data.drop_duplicates()

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78233 entries, 0 to 78255
Data columns (total 9 columns):
age_upon_outcome    78233 non-null int64
animal_id           78233 non-null object
animal_type         78233 non-null object
breed               78233 non-null object
color               78233 non-null object
date_of_birth       78233 non-null object
outcome_subtype     50308 non-null object
outcome_type        78221 non-null object
sex_upon_outcome    78231 non-null object
dtypes: int64(1), object(8)
memory usage: 6.0+ MB
